In [1]:
from utils.tushare_utils  import pro,ts
df = pro.stock_basic(
                exchange='',
                list_status='L',
                fields='ts_code,name'
            )
name = df[df['ts_code']=='000002.SZ']['name'][1]
print(ts.realtime_quote(ts_code='600000.SH', src='dc'))

没有找到 trade_date 为 20250829 的记录
   NAME    TS_CODE      DATE       TIME   OPEN  PRE_CLOSE  PRICE   HIGH  \
0  浦发银行  600000.SH  20250829   16:11:42  13.81      13.86  13.63  13.99   

     LOW    BID  ...  A1_V   A1_P    A2_V   A2_P  A3_V  A3_P    A4_V   A4_P  \
0  13.63  13.63  ...  30.0  13.68  2387.0  13.69  72.0  13.7  2469.0  13.71   

   A5_V   A5_P  
0  50.0  13.72  

[1 rows x 33 columns]


In [ ]:
from utils.tushare_utils import IndexAnalysis
import mplfinance as mpf
import pandas as pd
import numpy as np

# 获取股票 002594.SZ 的 1分钟频率数据
df = IndexAnalysis.rt_min('002594.SZ', 5)
print("原始数据列名:", df.columns.tolist())
print(df.head())

# 数据预处理
# 1. 确保时间列是datetime类型，并设置为索引
df['candle_end_time'] = pd.to_datetime(df['candle_end_time'])

# 2. 控制显示的K线数量
k_bars_to_show = 100  # 控制显示的K线数量
df = df.iloc[-k_bars_to_show:] if len(df) > k_bars_to_show else df
df.set_index('candle_end_time', inplace=True)

# 3. 检查并处理列名以符合mplfinance要求
if 'volume' in df.columns:
    df.rename(columns={'volume': 'Volume'}, inplace=True)
elif 'amount' in df.columns:
    print("使用 'amount' (成交额) 列作为成交量进行绘图。")
    df.rename(columns={'amount': 'Volume'}, inplace=True)
else:
    print("数据中未找到 'volume' 或 'amount' 列，将不绘制成交量。")

# 重命名价格列
df.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close'
}, inplace=True)

# 4. 计算移动平均线
# 计算5周期和20周期移动平均线
df['MA5'] = df['Close'].rolling(window=5, min_periods=1).mean()
df['MA20'] = df['Close'].rolling(window=20, min_periods=1).mean()

print("处理后的数据列名:", df.columns.tolist())
print(df.tail())

# 5. 创建移动平均线的附加图
apds = [
    mpf.make_addplot(df['MA5'], color='blue', width=1.0),
    mpf.make_addplot(df['MA20'], color='orange', width=1.0)
]

# 6. 绘制K线图
# 判断是否有成交量数据
has_volume = 'Volume' in df.columns

# 创建自定义样式
style = mpf.make_mpf_style(
    marketcolors=mpf.make_marketcolors(
        up='red',
        down='green',
        edge='inherit',
        wick='inherit',
        volume='in'
    ),
    gridstyle=':',
    gridcolor='lightgray',
    facecolor='white'
)

# 绘制图表
mpf.plot(
    df,
    type='candle',
    style=style,
    title=f'002594.SZ 1分钟K线图 (最近{len(df)}根K线)',
    ylabel='价格',
    volume=has_volume,
    addplot=apds,
    figratio=(12, 8),
    figscale=1.2,
    datetime_format='%H:%M',
    xrotation=45,
    show_nontrading=False
)

In [18]:
from utils.tushare_utils import IndexAnalysis


import pandas as pd
import numpy as np

import pandas as pd
import numpy as np
def calculate_rsi_tonghuashun(data, period=14):
    """
    计算与同花顺匹配的RSI值（使用Wilder平滑方法）
    
    参数:
    data: DataFrame, 包含价格数据，必须有'close'列
    period: int, RSI计算周期（6或12）
    
    返回:
    float: 最新一条数据的RSI值
    """
    # 确保有足够的数据点
    if len(data) < period + 1:
        return None
    
    # 复制数据以避免修改原始DataFrame
    df = data.copy()
    
    # 计算价格变化
    delta = df['close'].diff()
    
    # 分离上涨和下跌
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    # 计算初始平均值（前period个周期的简单平均）
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    # 使用Wilder平滑方法计算后续平均值
    for i in range(period, len(df)):
        avg_gain.iloc[i] = (avg_gain.iloc[i-1] * (period - 1) + gain.iloc[i]) / period
        avg_loss.iloc[i] = (avg_loss.iloc[i-1] * (period - 1) + loss.iloc[i]) / period
    
    # 计算相对强度(RS)
    rs = avg_gain / avg_loss
    
    # 计算RSI
    rsi = 100 - (100 / (1 + rs))
    
    return rsi.iloc[-1]  # 返回最新值

def get_tonghuashun_rsi_values(data):
    """
    计算同花顺风格的RSI6和RSI12值
    
    参数:
    data: DataFrame, 包含价格数据，必须有'close'列
    
    返回:
    dict: 包含RSI6和RSI12值的字典
    """
    # 计算RSI6
    rsi6 = calculate_rsi_tonghuashun(data, period=6)
    
    # 计算RSI12
    rsi12 = calculate_rsi_tonghuashun(data, period=12)
    
    return {
        "RSI6": round(rsi6, 2) if rsi6 is not None else None,
        "RSI12": round(rsi12, 2) if rsi12 is not None else None
    }
print(get_tonghuashun_rsi_values(IndexAnalysis.rt_min('399006.SZ',30)))

{'RSI6': 23.9, 'RSI12': 32.48}


In [11]:
from utils.date_utils import Date_utils
from utils.tushare_utils import ts
from utils.GetStockData import result
filtered_df = result[
    (result['total_mv'] > 500) & 
    (result['ts_code'].astype(str).str.startswith(('60', '00')))
]
# print(filtered_df)

print(ts.pro_bar(ts_code='159742.SZ',asset = 'FD', start_date=Date_utils.get_date_by_step(Date_utils.get_today(replace=True),-5), end_date=Date_utils.get_today(replace=True), ma=[5,10, 20, 30,60,120]))

没有找到 trade_date 为 20250905 的记录
        ts_code trade_date  pre_close   open   high    low  close  change  pct_chg         vol  ...  ma10     ma_v_10  ma20     ma_v_20  ma30     ma_v_30  ma60     ma_v_60  ma120    ma_v_120
0     159742.SZ   20250905      0.763  0.764  0.777  0.763  0.776   0.013   1.7038  4691041.10  ...  0.78  6809128.30  0.77  5780752.83  0.76  5043642.54  0.75  3901232.31   0.74  3948814.52
1     159742.SZ   20250904      0.774  0.778  0.781  0.759  0.763  -0.011  -1.4212  5866382.00  ...  0.78  6979577.89  0.77  5683971.87  0.76  5005339.00  0.75  3883292.78   0.74  3955921.52
2     159742.SZ   20250903      0.782  0.787  0.790  0.773  0.774  -0.008  -1.0230  5277424.01  ...  0.78  6826202.70  0.77  5539498.95  0.76  4919298.77  0.75  3843531.26   0.74  3952594.28
3     159742.SZ   20250902      0.789  0.789  0.791  0.777  0.782  -0.007  -0.8872  5468322.00  ...  0.78  6827599.70  0.77  5419020.40  0.76  4882919.54  0.74  3801813.61   0.74  3967504.68
4     159742.S